In [378]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbtpro as vbt
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time
import helpers as pth
from numba import njit
import talib

theme = "light"
vbt.settings.set_theme(theme)

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]
plt.style.use("classic" if theme == "light" else "dark_background")

if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

In [379]:
%%html
<style>
.dataframe {
    font-size: 9pt; /* Adjust font size as needed */
}
</style>

In [380]:
symbols = ["MES", "6B"]
df = pth.SierraChartData.ffill(
    pth.SierraChartData.pull(
        symbols,
        timeframe="1min",
        start="2024-11-01",
        end="2024-12-01",
    )
)
df

100%|##########| 2/2 [00:02<00:00,  1.39s/it, symbol=6B]

/Users/e1211913/miniconda3/envs/trading/lib/python3.12/site-packages/vectorbtpro/data/base.py:1728: VBTWarning: Symbols have mismatching index. Setting missing data points to NaN.


In [381]:
close = df.data["MES"].close
close.vbt.plot()

FigureWidget({
    'data': [{'name': 'close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'daa3d0e7-3c4f-4d03-bb92-08c14be6d1fa',
              'x': array([datetime.datetime(2024, 11, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 11, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([5808.5 , 5808.75, 5808.5 , ..., 6115.75, 6116.  , 6115.75])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'norm

### Generators

In [382]:
df_small = pth.SierraChartData.ffill(
    pth.SierraChartData.pull(
        symbols,
        timeframe="1min",
        start="2024-12-04 13:30",
        end="2024-12-04 20:30",
    )
)
close = df_small.get("close")
close["MES"].vbt.plot()

100%|##########| 2/2 [00:02<00:00,  1.37s/it, symbol=6B]

FigureWidget({
    'data': [{'name': 'MES',
              'showlegend': True,
              'type': 'scatter',
              'uid': '2ba679a1-1bae-4a58-8949-1659c0cc7cb4',
              'x': array([datetime.datetime(2024, 12, 4, 13, 30, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 13, 31, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 13, 32, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2024, 12, 4, 20, 28, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 20, 29, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 20, 30, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([6151.  , 6150.75, 6150.75, ..., 6167.  , 6166.75, 6166.  ])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
               

### Simple Example

In [383]:
@njit(nogil=True)
def place_func_nb(c):
    last = -1
    for i in range(c.out.shape[0]):
        if i % 50 == 0:
            c.out[i] = True
            last = i
    return last


symbol_wrapper = df_small.get_symbol_wrapper()
mask = vbt.pd_acc.signals.generate(
    symbol_wrapper.shape,
    place_func_nb,
    wrapper=symbol_wrapper,
)
mask.sum()

symbol
MES    9
6B     9
dtype: int64

In [384]:
pf = vbt.Portfolio.from_signals(
    close,
    mask,
    mask.shift(49).dropna().astype(bool),
)
pf["MES"].plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '06c93b12-eaf3-4134-a444-5969aef27716',
              'x': array([datetime.datetime(2024, 12, 4, 13, 30, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 13, 31, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 13, 32, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2024, 12, 4, 20, 28, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 20, 29, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 20, 30, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([100., 100., 100., ..., 

### Time Based Example

In [385]:
@njit
def place_time_func_nb(c, index):
    last = -1
    prev_hour = vbt.dt_nb.hour_nb(index[0])
    for i in range(c.out.shape[0]):
        # place a trade every hour
        hour = vbt.dt_nb.hour_nb(index[i])
        if hour != prev_hour:
            c.out[i] = True
            last = i
        prev_hour = hour
    return last


mask = vbt.pd_acc.signals.generate(
    symbol_wrapper.shape,
    place_time_func_nb,
    vbt.dt.to_ns(symbol_wrapper.index),
    wrapper=symbol_wrapper,
)
mask.sum()

symbol
MES    7
6B     7
dtype: int64

In [386]:
pf = vbt.Portfolio.from_signals(
    close,
    mask,
    mask.shift(59).dropna().astype(bool),
)
pf["MES"].plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'mode': 'lines',
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'd273abbf-5b1c-43bb-a703-2fb6330c6c57',
              'x': array([datetime.datetime(2024, 12, 4, 13, 30, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 13, 31, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 13, 32, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2024, 12, 4, 20, 28, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 20, 29, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 20, 30, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([6151.  , 6150.7

### Signal Factory

In [389]:
@njit
def place_func_entries_nb(c, every, index):
    last = -1
    for i in range(c.out.shape[0]):
        if i % every == 0:
            c.out[i] = True
            last = i
    return last


EntryGenerator = vbt.SignalFactory(
    mode="entries",
    param_names=["every"],
).with_place_func(
    entry_place_func_nb=place_func_entries_nb,
    entry_settings=dict(
        pass_params=["every"],
    ),
    var_args=True,
)

entry_generator = EntryGenerator.run(
    symbol_wrapper.shape,
    np.arange(5, 61, 5),
    vbt.dt.to_ns(symbol_wrapper.index),
    input_index=symbol_wrapper.index,
    input_columns=symbol_wrapper.columns,
)
entry_generator.entries.sum().vbt.barplot(title="# trades per period and instrument")

FigureWidget({
    'data': [{'showlegend': False,
              'type': 'bar',
              'uid': '5ee897d9-a404-4abe-89d1-1d9cb968184b',
              'x': [(5, 'MES'), (5, '6B'), (10, 'MES'), (10, '6B'), (15, 'MES'),
                    (15, '6B'), (20, 'MES'), (20, '6B'), (25, 'MES'), (25, '6B'),
                    (30, 'MES'), (30, '6B'), (35, 'MES'), (35, '6B'), (40, 'MES'),
                    (40, '6B'), (45, 'MES'), (45, '6B'), (50, 'MES'), (50, '6B'),
                    (55, 'MES'), (55, '6B'), (60, 'MES'), (60, '6B')],
              'y': array([85, 85, 43, 43, 29, 29, 22, 22, 17, 17, 15, 15, 13, 13, 11, 11, 10, 10,
                           9,  9,  8,  8,  8,  8])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'ma

In [413]:
# TODO: continue here when IF you're coming back to vectorised signal generation.